In [1]:
import os
import sys

# Comprueba si el código se está ejecutando en Google Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

path_absolute = ''
if IN_COLAB:
    print("El código se está ejecutando en Google Colab.")
    from google.colab import drive
    import tensorflow as tf
    print("Versión de TensorFlow:", tf.__version__)
    print("Dispositivos disponibles:", tf.config.list_physical_devices())

    drive.mount('/content/drive')
    path_absolute = '/content/drive/Othercomputers/Mi_portátil/TFM/WorkSpace/DATASET TFM'

    path_workspace = '/content/drive/Othercomputers/Mi_portátil/TFM/WorkSpace/'

    # Cambia al directorio de tu carpeta en Google Drive
    os.chdir(path_absolute)

    # Lista los archivos y carpetas en el directorio actual
    contenido_carpeta = os.listdir(path_absolute)
    print("Contenido de la carpeta en Google Drive:")
    print(contenido_carpeta)

    # Verificar la GPU
    device_name = tf.test.gpu_device_name()
    if device_name != '/device:GPU:0':
        raise SystemError('GPU no encontrada')
    print(f'Encontrada GPU: {device_name}')

    # Habilitar la GPU para TensorFlow
    physical_devices = tf.config.list_physical_devices('GPU')
    if len(physical_devices) > 0:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        print('Memoria de la GPU configurada dinámicamente')
    else:
        print('No se encontraron dispositivos GPU configurables')
else:
    print("El código se está ejecutando en un entorno local.")
    path_workspace ='C:/Users/jaime/OneDrive - Universidad de Málaga/Escritorio/UNIR/TFM/WorkSpace/'
    path_absolute = os.getcwd().replace("\\", "/")
    path_absolute = 'C:/Users/jaime/OneDrive - Universidad de Málaga/Escritorio/UNIR/TFM/WorkSpace/DATASET TFM'

datasets_path = "/ml-25m/"
path_absolute = path_absolute+datasets_path


sys.path.append(path_workspace)

El código se está ejecutando en un entorno local.


En este dataframe se hace un procesado de los conjuntos de datos genome-tags.csv y genome-scores.csv.

**genome-tags.csv** --> Cada entrada representa la relevancia de un cierto tag (tagId) para una película específica (movieId). Por ejemplo, una entrada puede ser: movieId, tagId, relevance. Aquí, tagId es un identificador que hace referencia a un cierto tipo de característica o atributo asociado a una película.

**genome-scores.csv** --> Cada entrada proporciona una descripción o etiqueta para un cierto tagId. Por ejemplo, una entrada puede ser: tagId, tag. Aquí, tagId es el mismo identificador que se encuentra en el archivo genome-scores.csv, y tag es una etiqueta descriptiva que explica qué tipo de característica o atributo representa ese tagId.

El parámetro "relevance" en el contexto del archivo "genome-scores.csv" representa la relevancia de un determinado tag (atributo o característica) para una película específica. Este valor de relevancia indica qué tan fuertemente una película exhibe cierta propiedad o característica representada por el tag correspondiente.


Se ha elejido los 5 tags mas relevantes por pelicula y se ha fusionado con el conjunto de datos de genome tags para tener una clumna con los 5 tas más relevantes por pelicula. 

In [2]:
import pandas as pd

In [3]:
genome_tags = pd.read_csv(path_absolute+'genome-tags.csv', sep=',')
genome_tags_scores = pd.read_csv(path_absolute+'genome-scores.csv', sep=',')

In [4]:
print(genome_tags.shape)
genome_tags.head()

(1128, 2)


,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


In [5]:
num_unique_tagId = genome_tags['tagId'].nunique()
print("Número de valores únicos en la columna tagId:", num_unique_tagId)

Número de valores únicos en la columna tagId: 1128


In [6]:
print(genome_tags_scores.shape)
genome_tags_scores.head()

(15584448, 3)


,movieId,tagId,relevance
0,1,1,0.02875
1,1,2,0.02375
2,1,3,0.06250
3,1,4,0.07575
4,1,5,0.14075


In [7]:
# Contar el número de valores únicos en la columna movieId
num_unique_movieId = genome_tags_scores['movieId'].nunique()
print("Número de valores únicos en la columna movieId:", num_unique_movieId)

Número de valores únicos en la columna movieId: 13816


In [8]:
top_5_genome_relevance = genome_tags_scores.groupby('movieId').apply(lambda x: x.nlargest(5, 'relevance')).reset_index(drop=True)


In [9]:
top_5_genome_relevance.head(10)

,movieId,tagId,relevance
0,1,1036,0.99925
1,1,244,0.99875
2,1,786,0.99575
3,1,589,0.98575
4,1,64,0.98425
5,2,29,0.97600
6,2,204,0.95900
7,2,377,0.95175
8,2,588,0.95000
9,2,951,0.93200


In [10]:
top_5_genome_relevance.shape

(69080, 3)

In [11]:
merged_df = pd.merge(genome_tags, top_5_genome_relevance, on='tagId', suffixes=('_df1', '_df2'))

In [12]:
merged_df.head(100000)

,tagId,tag,movieId,relevance
0,1,007,10,0.99950
1,1,007,1722,0.99975
2,1,007,2376,0.99975
3,1,007,2947,0.99950
4,1,007,2948,0.99975
...,...,...,...,...
69075,1128,zombies,160112,0.76125
69076,1128,zombies,160674,0.98975
69077,1128,zombies,162082,0.98250
69078,1128,zombies,164367,0.97725


In [13]:
merged_df.shape

(69080, 4)

In [14]:
genome_tags_scores_without_relevance = top_5_genome_relevance.drop('relevance', axis=1)

In [15]:
genome_tags_scores_without_relevance.head()

,movieId,tagId
0,1,1036
1,1,244
2,1,786
3,1,589
4,1,64


In [16]:
genome_tags_scores_without_relevance.shape

(69080, 2)

In [17]:
merged_df_2 = pd.merge(genome_tags, genome_tags_scores_without_relevance, on='tagId', suffixes=('_df1', '_df2'))

In [18]:
merged_df_2.head(100)

,tagId,tag,movieId
0,1,007,10
1,1,007,1722
2,1,007,2376
3,1,007,2947
4,1,007,2948
...,...,...,...
95,3,18th century,5321
96,3,18th century,5365
97,3,18th century,5646
98,3,18th century,5686


In [19]:
merged_df_3 = merged_df_2.drop('tagId', axis=1)

In [20]:
merged_df_3.shape

(69080, 2)

In [21]:
# Agrupar por 'movieId' y concatenar los valores de 'tag' separados por coma
df = merged_df_3.groupby('movieId')['tag'].agg(lambda x: ', '.join(x)).reset_index()
df['tag'] = df['tag'].apply(lambda x: ', '.join(set(x.split(', '))))

In [22]:
df.head()

,movieId,tag
0,1,"animation, kids and family, pixar animation, t..."
1,2,"children, adventure, fantasy, kids, special ef..."
2,3,"comedy, original, sequel, sequels, good sequel"
3,4,"women, girlie movie, divorce, romantic, chick ..."
4,5,"father daughter relationship, sequel, sequels,..."


In [23]:
df.shape

(13816, 2)

In [24]:
# df.to_csv(path_absolute+'movies_genome_tags.csv', index=False)

------------------------------------------------------------------------------------------------------

In [25]:
df_movies_userTag= pd.read_csv(path_absolute+'tags.csv')

In [26]:
print(df_movies_userTag.shape)
df_movies_userTag.head()

(1093360, 4)


,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,dark comedy,1573943598
3,4,1732,great dialogue,1573943604
4,4,7569,so bad it's good,1573943455


In [27]:
# Contar el número de valores únicos en la columna movieId
num_unique_userID = df_movies_userTag['userId'].nunique()
num_unique_movieId = df_movies_userTag['movieId'].nunique()
num_unique_tagId = df_movies_userTag['tag'].nunique()

print("Número de valores únicos en la columna tagId:", num_unique_tagId)
print("Número de valores únicos en la columna userId:", num_unique_userID)
print("Número de valores únicos en la columna movieId:", num_unique_movieId)

Número de valores únicos en la columna tagId: 73050
Número de valores únicos en la columna userId: 14592
Número de valores únicos en la columna movieId: 45251


In [28]:
# Cuenta los valores nulos en cada columna del DataFrame
valores_nulos_por_columna = df_movies_userTag.isnull().sum()
# Cuenta los valores nulos en todo el DataFrame
total_valores_nulos = df_movies_userTag.isnull().sum().sum()
# Imprime la cantidad de valores nulos por columna
print("Valores nulos por columna:")
print(valores_nulos_por_columna)
# Imprime el total de valores nulos en el DataFrame
print("\nTotal de valores nulos en el dataset:", total_valores_nulos)

Valores nulos por columna:
userId        0
movieId       0
tag          16
timestamp     0
dtype: int64

Total de valores nulos en el dataset: 16


In [29]:
sub_df_movies_userTag = df_movies_userTag[['movieId', 'tag']]

In [30]:
print(sub_df_movies_userTag.shape)
sub_df_movies_userTag.head()

(1093360, 2)


,movieId,tag
0,260,classic
1,260,sci-fi
2,1732,dark comedy
3,1732,great dialogue
4,7569,so bad it's good


In [31]:
# Contar el número de valores únicos en la columna movieId
num_unique_movieId = sub_df_movies_userTag['movieId'].nunique()
num_unique_tagId = sub_df_movies_userTag['tag'].nunique()

print("Número de valores únicos en la columna tagId:", num_unique_tagId)
print("Número de valores únicos en la columna movieId:", num_unique_movieId)

Número de valores únicos en la columna tagId: 73050
Número de valores únicos en la columna movieId: 45251


In [32]:
sub_df_movies_userTag_groupTags = sub_df_movies_userTag.groupby('movieId')['tag'].agg(lambda x: ','.join(str(i) for i in x.fillna(''))).reset_index()

In [33]:
# Función para eliminar duplicados en una cadena separada por comas
def remove_duplicates(tags):
    unique_tags = list(set(tags.split(',')))
    return ', '.join(unique_tags)

# Aplicar la función a la columna 'tag'
sub_df_movies_userTag_groupTags['tag'] = sub_df_movies_userTag_groupTags['tag'].apply(remove_duplicates)


In [34]:
print(sub_df_movies_userTag_groupTags.shape)
sub_df_movies_userTag_groupTags.head()

(45251, 2)


,movieId,tag
0,1,"friends, boy, avi, animated, very good, 55 mov..."
1,2,"Adaptation of Book, adventure, Children, fanta..."
2,3,"Walter Matthau, Burgess Meredith, old man, Sop..."
3,4,"based on novel or book, characters, CLV, inter..."
4,5,"baby, daughter, Diane Keaton, wedding, contrac..."


In [35]:
# Contar el número de valores únicos en la columna movieId
num_unique_movieId = sub_df_movies_userTag_groupTags['movieId'].nunique()
num_unique_tagId = sub_df_movies_userTag_groupTags['tag'].nunique()

print("Número de valores únicos en la columna tagId:", num_unique_tagId)
print("Número de valores únicos en la columna movieId:", num_unique_movieId)

Número de valores únicos en la columna tagId: 40242
Número de valores únicos en la columna movieId: 45251


In [36]:
# Cuenta los valores nulos en cada columna del DataFrame
valores_nulos_por_columna = sub_df_movies_userTag_groupTags.isnull().sum()
# Cuenta los valores nulos en todo el DataFrame
total_valores_nulos = sub_df_movies_userTag_groupTags.isnull().sum().sum()
# Imprime la cantidad de valores nulos por columna
print("Valores nulos por columna:")
print(valores_nulos_por_columna)
# Imprime el total de valores nulos en el DataFrame
print("\nTotal de valores nulos en el dataset:", total_valores_nulos)

Valores nulos por columna:
movieId    0
tag        0
dtype: int64

Total de valores nulos en el dataset: 0


In [37]:
sub_df_movies_user_tag = df_movies_userTag.copy()
# Suponiendo que df es tu DataFrame y 'columna1', 'columna2' son los nombres de las columnas que deseas eliminar
sub_df_movies_user_tag = sub_df_movies_user_tag.drop(columns=['tag', 'timestamp'])

In [38]:
sub_df_movies_userTag_non_duplicated = sub_df_movies_user_tag[~sub_df_movies_user_tag.duplicated(subset=['userId', 'movieId'], keep='first')]

In [39]:
print(sub_df_movies_userTag_non_duplicated.shape)
sub_df_movies_userTag_non_duplicated.head()

(305356, 2)


,userId,movieId
0,3,260
2,4,1732
4,4,7569
5,4,44665
6,4,115569


In [40]:
# Contar el número de valores únicos en la columna movieId
num_unique_movieId = sub_df_movies_userTag_non_duplicated['movieId'].nunique()
num_unique_userId= sub_df_movies_userTag_non_duplicated['userId'].nunique()

print("Número de valores únicos en la columna userId:", num_unique_userId)
print("Número de valores únicos en la columna movieId:", num_unique_movieId)

Número de valores únicos en la columna userId: 14592
Número de valores únicos en la columna movieId: 45251


In [41]:
movies_tags_by_users = pd.merge(sub_df_movies_userTag_non_duplicated, sub_df_movies_userTag_groupTags, on='movieId', how='left', suffixes=('_df1', '_df2'))

In [42]:
print(movies_tags_by_users.shape)
movies_tags_by_users.head()

(305356, 3)


,userId,movieId,tag
0,3,260,", conflict, Special Effects, underdog story, N..."
1,4,1732,"drugs, Best Movie Ever, girls, Nudity (Full Fr..."
2,4,7569,"london england, misogyny, gyrocopter, secret b..."
3,4,44665,"KAF, To See, hitman, gangster, Rythm, Josh Har..."
4,4,115569,"television, thrilling, American Dream, depress..."


In [43]:
# Contar el número de valores únicos en la columna movieId
num_unique_movieId = movies_tags_by_users['movieId'].nunique()
num_unique_userId= movies_tags_by_users['userId'].nunique()

print("Número de valores únicos en la columna userId:", num_unique_userId)
print("Número de valores únicos en la columna movieId:", num_unique_movieId)

Número de valores únicos en la columna userId: 14592
Número de valores únicos en la columna movieId: 45251


In [44]:
# movies_tags_by_users.to_csv(path_absolute+'movies_tags_by_users.csv', index=False)